Detección de caras con webcam

In [1]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import numpy as np

Se ha realizado un filtro de incógnito


In [6]:
glasses = cv2.imread("Gafas.png", cv2.IMREAD_UNCHANGED)
#glasses = cv2.resize()
hat = cv2.imread("Sombrero.png", cv2.IMREAD_UNCHANGED)
mustache = cv2.imread("Mostacho.png", cv2.IMREAD_UNCHANGED)

def overlay_image(im1, im2, x_offset, y_offset):
    '''Mutates im1, placing im2 over it at a given offset.'''
    try:
        y1, y2 = y_offset, y_offset + im2.shape[0]
        x1, x2 = x_offset, x_offset + im2.shape[1]
        alpha_s = im2[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s

        for c in range(0, 3):
            im1[y1:y2, x1:x2, c] = (alpha_s * im2[:, :, c] +
                                    alpha_l * im1[y1:y2, x1:x2, c])
            
        return im1
    except:
        return im1
    
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

# Face detection and eye model setup
imodoF = 0
imodoE = 0

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    print(eyes)
    
    frame = overlay_image(frame, glasses, lex - 30, ley - 30)    
    frame = overlay_image(frame, hat, lex - 55 , ley - 150) 
    frame = overlay_image(frame, mustache, lex - 55 , ley + 40)  
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    # Face detector change
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    #Eye detector change
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
[359, 37, 366, 18]
[360, 38, 398, 39]
[359, 38, 399, 38]
[360, 37, 398, 37]
[359, 36, 367, 17]
[359, 37, 397, 41]
[360, 37, 367, 18]
[360, 36, 367, 17]
[359, 36, 367, 17]
[359, 36, 398, 40]
[360, 34, 400, 36]
[360, 31, 369, 14]
[360, 31, 399, 36]
[360, 31, 399, 35]
[358, 32, 400, 36]
[358, 39, 400, 44]
[356, 53, 402, 57]
[356, 55, 402, 60]
[357, 66, 402, 70]
[354, 74, 402, 79]
[353, 82, 401, 85]
[354, 83, 402, 87]
[353, 88, 402, 92]
[354, 93, 404, 97]
[355, 98, 405, 101]
[355, 99, 406, 103]
[358, 105, 408, 108]
[359, 109, 410, 113]
[361, 112, 412, 117]
[362, 112, 412, 116]
[364, 114, 415, 118]
[364, 115, 415, 118]
[366, 114, 416, 119]
[368, 113, 418, 118]
[368, 113, 417, 118]
[369, 112, 418, 116]
[368, 111, 419, 116]
[370, 110, 422, 115]
[371, 110, 421, 116]
[377, 106, 427, 111]
[378, 105, 429, 110]
[385, 100, 432, 106]
[386, 94, 434, 100]
[387, 94, 435, 99]
[386, 101, 434, 106]
[384, 119, 432, 124]
[380, 142, 429, 146]
[379, 147, 427, 152]
[374, 167, 425, 171]
[371, 183, 423,